To generate synthetic VisiumHD data from Xenium, please read and run all the cells below. Thanks!

## Download Xenium output from 10X website
Paste the URL for the binned_outputs.tar.gz for the sample you want to analyze.

1. Go to Xenium public datasets page:https://www.10xgenomics.com/datasets?query=&page=1&configure%5BhitsPerPage%5D=50&configure%5BmaxValuesPerFacet%5D=1000&refinementList%5Bproduct.name%5D%5B0%5D=In%20Situ%20Gene%20Expression&refinementList%5Bspecies%5D%5B0%5D=Human&refinementList%5BdiseaseStates%5D%5B0%5D=colorectal%20cancer

2. Select sample to analyze scrolling down to downloads section, click "Batch download"


In [ ]:
import zipfile
xenium_outputs_url = "https://cf.10xgenomics.com/samples/xenium/2.0.0/Xenium_V1_Human_Colorectal_Cancer_Addon_FFPE/Xenium_V1_Human_Colorectal_Cancer_Addon_FFPE_outs.zip"
# Step 1: Download the raw Xenium output
!curl -O {xenium_outputs_url}

# Extract the ZIP file
zip_file_path = xenium_outputs_url.split("/")[-1]

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall("extracted_files")

print("Extraction completed.")

### Install prerequisite libraries

In [ ]:
!pip install --upgrade pip
!pip install scipy
!pip install shapely
!pip install tifffile
!pip install plotly
!pip install tensorflow-gpu==2.10.0
!pip install stardist
!pip install geopandas
!pip install scanpy
!pip install fastparquet
!pip install opencv-python
!pip install geojson
!pip install scikit-learn

### Import Relevant Libraries

In [ ]:
import geopandas as gpd # Geopandas for storing Shapely objects
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import scanpy as sc
import pandas as pd
from scipy import sparse
import anndata
import os
import gzip
import numpy as np
import re
import shapely
from shapely.geometry import Polygon, Point # Representing bins and cells as Shapely Polygons and Point objects
from shapely import wkt

### Load Cell & Transcripts Info

In [ ]:
# Load the transcript data
transcripts_path = "extracted_files/transcripts.csv.gz"
with gzip.open(transcripts_path, 'rt') as f:
    transcripts_df = pd.read_csv(f)

# Load cell info
cells_path = "extracted_files/cells.csv.gz"
with gzip.open(cells_path, 'rt') as f:
    cells_df = pd.read_csv(f)


### Load Cell Boundary Info

In [ ]:
import zarr

zarr_file = zarr.open('extracted_files/cells.zarr.zip', mode='r')
print(zarr_file.tree())

In [ ]:
file = zarr_file['polygon_sets/0/vertices'][:]
# 1 is whole cell, 0 is nucleus

### Create folders to store synthetic data

For both the `seqfish_dir` and `enact_data_dir`, change `"/home/oneai/"` to the directory that stores this repo.

In [ ]:
xenium_dir = "/home/oneai/oneai-dda-spatialtr-visiumhd_analysis/synthetic_data/xenium" # Update it to the directory where you want to save the synthetic data
enact_data_dir = "/home/oneai/oneai-dda-spatialtr-visiumhd_analysis/cache/xenium_nuclei/chunks" # Directory that saves all the input and results of the enact pipeline, 
# should end with "oneai-dda-spatialtr-visiumhd_analysis/cache/seqfish/chunks"

transcripts_df_chunks_dir = os.path.join(xenium_dir, "transcripts_patches") # Directory to store the files that contain the transcripts info for each chunk
output_dir = os.path.join(enact_data_dir, "bins_gdf") # Directory to store the results of gene-to-bin assignment for each chunk
cells_df_chunks_dir =  os.path.join(enact_data_dir,"cells_gdf") 
ground_truth_dir =  os.path.join(xenium_dir, "ground_truth_nuclei")

# Making relevant directories
os.makedirs(xenium_dir, exist_ok=True)
os.makedirs(enact_data_dir, exist_ok=True)
os.makedirs(transcripts_df_chunks_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)
os.makedirs(cells_df_chunks_dir, exist_ok=True)
os.makedirs(ground_truth_dir, exist_ok=True)

### Generate Synthetic VesiumHD Dataset

#### Break transcripts df to patches (based on location)

Break transcripts df to patches of size 1000um x 1000um (larger patch size may result in memory issue)

In [ ]:
# patch size: 1000 um x 1000 um

patch_size = 1000

# patch indices
transcripts_df['x_patch'] = (transcripts_df['x_location'] // patch_size).astype(int)
transcripts_df['y_patch'] = (transcripts_df['y_location'] // patch_size).astype(int)
transcripts_df["patch_id"] = transcripts_df["x_patch"].astype(str) + "_" + transcripts_df["y_patch"].astype(str)

# Create a df for each patch
grouped = transcripts_df.groupby(['x_patch', 'y_patch'])
for (x_patch, y_patch), group in grouped:
    # Calculate the start and end locations for each patch
    # x_start = x_patch * patch_size
    # x_end = (x_patch + 1) * patch_size
    # y_start = y_patch * patch_size
    # y_end = (y_patch + 1) * patch_size
    
    filename = f"patch_{x_patch}_{y_patch}.csv"
    output_loc = os.path.join(transcripts_df_patch_dir , filename)
    group.to_csv(output_loc)

    print(f"Saved {filename}")

#### Generate synthetic vesiumHD for each patch

Each patch is broken into bins of size 2um x 2um. The synthetic data contains transcript counts orgnized by bin_id. Each row contains transcript counts for a unique bin. Bins with no transcript counts is not included. 

In addition to all the gene features, there are two additional columns represent the row number and column number of the bin, and a column contains the Shapely polygon item that represents the bin. The first column is the bin_id.

In [ ]:
def generate_synthetic_VesiumHD_data(transcripts_df, bin_size=2, whole_cell=True, QScore20=True):
    filtered_df = transcripts_df.copy()
    # only count transcripts in the nucleus
    if not whole_cell:
        filtered_df = transcripts_df[transcripts_df['overlaps_nucleus'] == 1].copy()
    
    only count transcripts with QScore >= 20
    if QScore20:
        filtered_df = filtered_df[filtered_df['qv'] >= 20].copy()
 
    # assigne bin to each transcript
    filtered_df.loc[:, 'row'] =np.ceil(filtered_df['y_location'] / bin_size).astype(int)
    filtered_df.loc[:, 'column'] = np.ceil(filtered_df['x_location'] / bin_size).astype(int)
    filtered_df.loc[:, 'assigned_bin_id'] = filtered_df.apply(
        lambda row: f"{bin_size}um_" + str(row['row']).zfill(5) +"_"+ str(row['column']).zfill(5),
        axis=1)
    
    bin_coordinates = filtered_df[['assigned_bin_id', 'row', 'column']].drop_duplicates().set_index('assigned_bin_id')
    bin_gene_matrix = filtered_df.groupby(['assigned_bin_id', 'feature_name']).size().unstack(fill_value=0)
    bin_gene_matrix_with_coords = bin_gene_matrix.merge(bin_coordinates, left_index=True, right_index=True)
    
    return bin_gene_matrix_with_coords

In [ ]:
# Extract row and column number from the bin_id
def extract_numbers(entry):
    match = re.search(r'_(\d{5})_(\d{5})', entry)
    if match:
        number1 = int(match.group(1).lstrip('0'))  
        number2 = int(match.group(2).lstrip('0'))  
        return number2*2-1, number1*2-1
    else:
        return None, None

In [ ]:
from tqdm import tqdm
def generate_bin_polys(bins_df, x_col, y_col, bin_size):
        """Represents the bins as Shapely polygons

        Args:
            bins_df (pd.DataFrame): bins dataframe
            x_col (str): column with the bin centre x-coordinate
            y_col (str): column with the bin centre y-coordinate
            bin_size (int): bin size in pixels

        Returns:
            list: list of Shapely polygons
        """
        geometry = []
        # Generates Shapely polygons to represent each bin

        if True:
            half_bin_size = bin_size / 2
            bbox_coords = pd.DataFrame(
                {
                    "min_x": bins_df[x_col] - half_bin_size,
                    "min_y": bins_df[y_col] - half_bin_size,
                    "max_x": bins_df[x_col] + half_bin_size,
                    "max_y": bins_df[y_col] + half_bin_size,
                }
            )
            geometry = [
                shapely.geometry.box(min_x, min_y, max_x, max_y)
                for min_x, min_y, max_x, max_y in tqdm(
                    zip(
                        bbox_coords["min_x"],
                        bbox_coords["min_y"],
                        bbox_coords["max_x"],
                        bbox_coords["max_y"],
                    ),
                    total=len(bins_df),
                )
            ]

        return geometry

In [ ]:
# Loop through all the transcripra_df chunks and generate gene-to-bin assignments 
patch_size = 1000
bin_size = 2
transcripts_df_chunks = os.listdir(transcripts_df_patch_dir)
for chunk_fname in transcripts_df_chunks:
    output_loc = os.path.join(output_dir, chunk_fname)
    # if os.path.exists(output_loc):
    #     continue
    if chunk_fname in [".ipynb_checkpoints"]:
        continue
    transcripts_df_chunk = pd.read_csv(os.path.join(transcripts_df_patch_dir, chunk_fname))
    bin_df_chunk = generate_synthetic_VesiumHD_data(transcripts_df_chunk, bin_size, whole_cell=True, QScore20=True)
    bin_df_chunk['column'] = bin_df_chunk['column']*2-1
    bin_df_chunk['row'] = bin_df_chunk['row']*2-1
    bin_df_chunk['geometry'] = generate_bin_polys(bin_df_chunk, 'column', 'row', 2)
    bin_gdf_chunk = gpd.GeoDataFrame( bin_df_chunk, geometry = bin_df_chunk['geometry'])
    bin_df_chunk.to_csv(output_loc)
    print(f"Successfully assigned transcripts to bins for {chunk_fname}")


### Generate cell_gdf as enact_pipeline input

This session generate the cell_df patches required to run the enact pipeline. The main purpose is to create Shapely polygons that represent the cell outline.

In [ ]:
def create_polygons(coords_array):
    polygons = []
    for row in coords_array:
        reshaped_coords = row.reshape(-1, 2)
        polygon = Polygon(reshaped_coords)
        polygons.append(polygon)
    return polygons

# Create the polygons
polygons = create_polygons(file)
cells_df['polygons'] = polygons

In [ ]:
cell_gdf_chunk = gpd.GeoDataFrame(cells_df, geometry = cells_df['polygons'])
cell_gdf_chunk.rename(columns={'x_centroid': 'cell_x', 'y_centroid': 'cell_y'}, inplace=True)
cell_gdf_chunk.drop("Unnamed: 0", axis=1, inplace=True)
cell_gdf_chunk[['cell_id','cell_x','cell_y','geometry']].to_csv(os.path.join(enact_data_dir, "cells_gdf"))

### Run ENACT bin-to-cell pipeline
In the configs.yaml file: 

    Set "analysis_name" in the configs.yaml file to "xenium" (or "xenium_nuclei).
    Set "run_synthetic" to True and all other steps to False.
    Set "bin_to_cell_method" to one of these four: "naive", "weighted_by_area", "weighted_by_gene", or "weighted_by_cluster"

Run `make run_enact`

### Generate Ground Truth

The following cell will generate and save the ground truth of the synthetic VisiumHD data for the use of bin-to-cell assignment methods evaluation. Ground truth dataframe consists of rows representing the transcript counts of each cell. Each column represents a gene feature (gene feature name is also the column name).

#### Generate Cell-gene matrix for evaluation

In [ ]:
def generate_ground_truth_table(transcripts_df, cells_df, whole_cell=True, QScore20=True, include_unassigned_transcript=False):
    filtered_df = transcripts_df
    
    # only count transcripts in the nucleus
    if not whole_cell:
        filtered_df = transcripts_df[transcripts_df['overlaps_nucleus'] == 1]
    
    # only count transcripts with QScore >= 20
    if QScore20:
        filtered_df = filtered_df[filtered_df['qv'] >= 20]
    
    # only count transcripts that are assigned to specific cells
    if not include_unassigned_transcript:
        filtered_df = filtered_df[filtered_df['cell_id'] != 'UNASSIGNED']
    
    pivot_df = filtered_df.pivot_table(index='cell_id', columns='feature_name', aggfunc='size', fill_value=0)
    
    merged_df = pivot_df.merge(cells_df[['cell_id']], left_index=True, right_on='cell_id', how='right')
    columns = ['cell_id'] + [col for col in merged_df.columns if col not in ['cell_id', 'x_centroid', 'y_centroid','polygons']]
    merged_df = merged_df[columns]
    merged_df.set_index('cell_id', inplace=True)
    #merged_df['total_gene_counts'] = merged_df.iloc[:, 3:].sum(axis=1)
    
    return merged_df

In [ ]:
bin_size = 2
cell_df_chunks = os.listdir(cells_df_chunks_dir)
for chunk_fname in cell_df_chunks:
    output_loc = os.path.join(ground_truth_dir,chunk_fname)
    if os.path.exists(output_loc):
        continue
    if chunk_fname in [".ipynb_checkpoints"]:
        continue
    cell_df_chunk = pd.read_csv(os.path.join(cell_dir, chunk_fname))
    groundtruth_chunk = generate_ground_truth_table(transcripts_df, cell_df_chunk, whole_cell=False, QScore20=False, include_unassigned_transcript=False)
    groundtruth_chunk.to_csv(output_loc)
    print(f"Successfully generated groundthuth for {chunk_fname}")

### Evaluation of ENACT bin-to-cell results

#### Overall precision, recall, and f1

Run this session with all the methods you have run with ENACT, change 'method' in the cell bellow to the one you want to evaluate.

In [ ]:
import pandas as pd
import numpy as np

method = "weighted_by_cluster"
results_dir = os.path.join(enact_data_dir, method, "bin_to_cell_assign")

# Initialize variables to accumulate weighted precision, recall, and F1
total_cells = 0
precision_sum = 0
recall_sum = 0
missing_cells_count = 0
total_cells_count = 0
results_chunks = os.listdir(results_dir)

for chunk_fname in results_chunks:
    if chunk_fname in [".ipynb_checkpoints"]:
        continue

    generated = pd.read_csv(os.path.join(results_dir, chunk_fname))
    ground_truth = pd.read_csv(os.path.join(ground_truth_dir, chunk_fname))
    if len(generated) ==0:
        print(chunk_fname)
        continue
    generated.rename(columns={'id': 'cell_id'}, inplace=True)
  
    # Align both dataframes by 'cell_id', filling missing cells in generated with 0
    merged = pd.merge(ground_truth, generated, on='cell_id', how='left', suffixes=('_gt', '_gen')).fillna(0)
    num_cells = (ground_truth.iloc[:, 1:] != 0).any(axis=1).sum()
    missing_cells_count += num_cells - len(generated)
    total_cells_count += num_cells

    ground_truth_aligned = merged.filter(like='_gt').values
    generated_aligned = merged.filter(like='_gen').values
    assert ground_truth_aligned.shape == generated_aligned.shape, "Aligned matrices must have the same shape!"

    num_cells = ground_truth_aligned.shape[0]

    # Compute precision for the current patch
    patch_precision = np.sum(np.minimum(generated_aligned, ground_truth_aligned)) / np.sum(generated_aligned)

    # Compute recall for the current patch
    patch_recall = np.sum(np.minimum(generated_aligned, ground_truth_aligned)) / np.sum(ground_truth_aligned)

    # F1 score for the current patch
    if patch_precision + patch_recall > 0:
        patch_f1 = 2 * (patch_precision * patch_recall) / (patch_precision + patch_recall)
    else:
        patch_f1 = 0

    # Accumulate the weighted precision, recall, and number of aligned cells
    precision_sum += patch_precision * num_cells
    recall_sum += patch_recall * num_cells
    total_cells += num_cells
    
#  Compute overall weighted precision, recall, and F1 score
overall_precision = precision_sum / total_cells
overall_recall = recall_sum / total_cells

if overall_precision + overall_recall > 0:
    overall_f1_score = 2 * (overall_precision * overall_recall) / (overall_precision + overall_recall)
else:
    overall_f1_score = 0 

# Print results
print(f"Overall Precision: {overall_precision}")
print(f"Overall Recall: {overall_recall}")
print(f"Overall F1 Score: {overall_f1_score}")
print(f"Total missing cells in the generated data compared to ground truth: {missing_cells_count}")
print(f"Total cells : {total_cells_count}")

#### Visualize the distribution using violin plots 

The following cells would create violin plots for all four methods in order to better compare the results. You can choose to only compare the ones you have run by changing the 'methods' list below to only include those.

In [ ]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt

# Define methods and their directories
methods = [
    {
        'name': 'Naive',
        'results_dir': os.path.join(enact_data_dir, "naive", "bin_to_cell_assign"),       
        'ground_truth_dir':ground_truth_dir
    },
    {
        'name': 'Weighted_by_area',
        'results_dir': os.path.join(enact_data_dir, "weighted_by_area", "bin_to_cell_assign"),       
        'ground_truth_dir':ground_truth_dir
    },
    {
        'name': 'Weighted_by_gene',
        'results_dir':  os.path.join(enact_data_dir, "weighted_by_gene", "bin_to_cell_assign"),      
        'ground_truth_dir': ground_truth_dir
    },
    {
        'name': 'Weighted_by_cluster',
        'results_dir':  os.path.join(enact_data_dir, "weighted_by_cluster", "bin_to_cell_assign"),   
        'ground_truth_dir': ground_truth_dir
    }
]

# Initialize a list to store per-patch metrics for all methods
metrics_list = []

# Loop through each method to compute per-patch metrics
for method in methods:
    method_name = method['name']
    results_dir = method['results_dir']
    ground_truth_dir = method['ground_truth_dir']
    
    print(f"Processing {method_name}...")
    
    # Get list of generated and ground truth files
    generated_files = [f for f in os.listdir(results_dir) if f.endswith('.csv') and f not in [".ipynb_checkpoints"]]
    ground_truth_files = [f for f in os.listdir(ground_truth_dir) if f.endswith('.csv') and f not in [".ipynb_checkpoints"]]
    
    # Find common files between generated results and ground truth
    common_files = set(generated_files) & set(ground_truth_files)
    
    if not common_files:
        print(f"No common files found for {method_name}. Skipping method.")
        continue
    
    # Loop through each common file (patch)
    for fname in common_files:
        ground_truth_path = os.path.join(ground_truth_dir, fname)
        generated_path = os.path.join(results_dir, fname)
        
        # Load ground truth and generated data
        ground_truth = pd.read_csv(ground_truth_path)
        generated = pd.read_csv(generated_path)
        
        # Skip if generated data is empty
        if generated.empty:
            print(f"No data in generated file {fname} for {method_name}. Skipping patch.")
            continue
        
        # Rename columns for consistency
        if 'id' in generated.columns:
            generated.rename(columns={'id': 'cell_id'}, inplace=True)
        
        # Merge ground truth and generated data on 'cell_id', filling missing values with 0
        merged = pd.merge(
            ground_truth, generated, on='cell_id', how='outer', suffixes=('_gt', '_gen')
        ).fillna(0)
        
        # Extract aligned matrices for ground truth and generated data
        ground_truth_aligned = merged.filter(regex='_gt$').values
        generated_aligned = merged.filter(regex='_gen$').values
        
        # Ensure matrices are aligned
        if ground_truth_aligned.shape != generated_aligned.shape:
            print(f"Shape mismatch in patch {fname} for {method_name}. Skipping patch.")
            continue
        
        # Compute counts for this patch
        tp = np.sum(np.minimum(generated_aligned, ground_truth_aligned))
        predicted = np.sum(generated_aligned)
        actual = np.sum(ground_truth_aligned)
        
        # Compute metrics for this patch
        precision = tp / predicted if predicted > 0 else 0
        recall = tp / actual if actual > 0 else 0
        f1_score = (
            2 * (precision * recall) / (precision + recall)
            if (precision + recall) > 0 else 0
        )
        
        # Store metrics for this patch
        metrics_list.append({
            'Method': method_name,
            'Patch': fname,
            'Precision': precision,
            'Recall': recall,
            'F1 Score': f1_score
        })

# Create a DataFrame with per-patch metrics
metrics_df = pd.DataFrame(metrics_list)

# Display the first few rows of the DataFrame
print("\nPer-Patch Metrics:")
print(metrics_df.head())

In [ ]:
# plotting
sns.set(style="whitegrid")

# Create a figure with subplots for each metric
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Precision Violin Plot
sns.violinplot(x='Method', y='Precision', data=metrics_df, ax=axes[0], inner='quartile', palette='Set2')
axes[0].set_title('Precision')
axes[0].set_xlabel('Method')
axes[0].set_ylabel('value')
axes[0].set_ylim(0,1)
axes[0].tick_params(axis='x', labelsize=8)  # Adjust the font size here

# Recall Violin Plot
sns.violinplot(x='Method', y='Recall', data=metrics_df, ax=axes[1], inner='quartile', palette='Set2')
axes[1].set_title('Recall')
axes[1].set_xlabel('Method')
axes[1].set_ylabel('value')
axes[1].set_ylim(0,1)
axes[1].tick_params(axis='x', labelsize=8)  # Adjust the font size here

# F1 Score Violin Plot
sns.violinplot(x='Method', y='F1 Score', data=metrics_df, ax=axes[2], inner='quartile', palette='Set2')
axes[2].set_title('F1 Score')
axes[2].set_xlabel('Method')
axes[2].set_ylabel('value')
axes[2].set_ylim(0,1)
axes[2].tick_params(axis='x', labelsize=8)  # Adjust the font size here

plt.tight_layout()
plt.show()